In [1]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws

In [2]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})

In [3]:
def sqlClause(termList, fc, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}' Or """.format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}'""".format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
    return owner_string

In [4]:
def sqlClause2(termList, fc, fieldName):
    owner_string = '('
    for owner in termList:
        if termList.index(owner) != len(termList)-1:
            owner_string = owner_string + "{owner}, ".format(owner = owner)
        elif termList.index(owner) == len(termList) -1 :
            owner_string = owner_string + "{owner}".format(owner = owner)
            
    owner_string = owner_string + ')'
    where_clause = """{field} IN {string}""".format(field = arcpy.AddFieldDelimiters(fc, fieldName), string = owner_string)
    return where_clause

In [5]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList
    

In [6]:
def narrowList(ogList):
    ogList = [i for i in ogList if not ('ESTATE' in i or 'SUSAN' in i or 'LLC' in i or 
                                                'COMPANY' in i or 'ASSOC' in i or 'ASSN' in i or
                                        'INC' in i or 'BANK' in i or 'CHURCH' in i or 'SOCIETY' in i or
                                               'RAILWAY' in i or 'PRESBYTERIAN' in i or 'CORP' in i or
                                       'FUND' in i)]
    
    removeList = [i for i in ogList if ('US' in i and not i.startswith('US'))]
    
    ogList = removeTerms(ogList, removeList)
    
    return ogList

In [7]:
def pubFind(myValues, searchTerms = ['USA', 'UNITED', 'STATE', 'OREGON', 'WASH', 'COUNTY', 'SKAMANIA', 'CITY', 'CDID', 'GOVERNMENT', 'SCHOOL', 'PUBLIC', 'PORT OF', 'METRO', 'DISTRICT']):
    pubLand = []
    myValues = [value.upper() for value in myValues]
    
    for owner in myValues:
        for term in searchTerms:
            if term in owner:
                pubLand.append(owner)
    
    
   
    return pubLand

In [8]:
#Completed Repair Geometry in ArcGis

In [9]:
# Create point feature class from county_merge
arcpy.FeatureToPoint_management('county_merge', 'parcelPoints', 'INSIDE')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\parcelPoints'>

In [10]:
# Create selection of public lands that intersects with Keith's original layer
# and spatial join together so we can get past ownership information
ownerOvrTime = arcpy.SelectLayerByLocation_management('parcelPoints', 'WITHIN', 'pub_pvt_exportpub_wm_Project')
arcpy.SpatialJoin_analysis(ownerOvrTime, 'pub_pvt_exportpub_wm_Project', 'ownerOverTime')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\ownerOverTime'>

In [11]:
# Select all the lands that do not meet the criteria for public ownership from the newly created layer
privLands = arcpy.SelectLayerByAttribute_management('ownerOverTime', where_clause = "PublicLand = 'F'")
arcpy.CopyFeatures_management(privLands, 'interimPrivate')


<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\interimPrivate'>

In [12]:
# Display unique owners in newly created layer
owners = unique_values('interimPrivate', 'Land_Owner')

In [13]:
# Search for publicly owned lands that did not meet the criteria
pubOwners = pubFind(owners)
pubOwners = narrowList(pubOwners)

In [14]:
print(pubOwners)

['ASTORIA CITY OF', 'BEACON HILL WATER/SEWER DISTRICT', 'CENTRAL CITY CONCERN', 'CITY OF CAMAS', 'CITY OF VANCOUVER', 'CITY OF WASHOUGAL', 'CITY OF WASHOUGAL', 'CLARK COUNTY PUBLIC WORKS', 'CLARK COUNTY PUBLIC WORKS', 'CLARK COUNTY PUBLIC WORKS OPERATIONS CENTER', 'CLARK COUNTY PUBLIC WORKS OPERATIONS CENTER', 'COL CO AD SCHOOL DIST 502-ST HELENS', 'COLUMBIA COUNTY HABITAT FOR HUMANITY', 'COLUMBIA COUNTY SHRINE CLUB', 'COWLITZ COUNTY CASA', 'COWLITZ COUNTY PUD', 'KALAMA PORT OF', 'LEXINGTON FLOOD CONTROL ZONE DISTRICT', 'LONGVIEW CITY OF', 'METRO & PORT OF PORTLAND-CI', 'METRO & PORT OF PORTLAND-CI', 'METRO (COLUMBIA PIONEER CEMETERY', 'METRO (GLEASON BOAT RAMP', 'METRO (POWELL GROVE CEMETERY', 'MULTNOMAH COUNTY DRAINAGE DISTRICT #1', 'MULTNOMAH COUNTY DRAINAGE DISTRICT #1', 'MULTNOMAH COUNTY LIBRARY DISTRICT', 'MULTNOMAH COUNTY LIBRARY DISTRICT', 'MULTNOMAH COUNTY SCHOOL DIST #3', 'MULTNOMAH COUNTY SCHOOL DIST #3', 'MULTNOMAH COUNTY SCHOOL DIST NO 3', 'MULTNOMAH COUNTY SCHOOL DIST NO 

In [15]:
removeList = ['CENTRAL CITY CONCERN','COLUMBIA COUNTY HABITAT FOR HUMANITY', 'COLUMBIA COUNTY SHRINE CLUB', 'COWLITZ COUNTY CASA',
             'FIRST ASSEMBLY OF GOD COWLITZ COUNTY', 'OCF JOSEPH E WESTON PUBLIC FOUNDATION', 'SAFE OF COLUMBIA COUNTY', 'UNITED STATES GYPSUM', 
              'UNITED STATES GYPSUM', 'UNITED STATES OF AMERICA BPA', 'UNITED STATES OF AMERICA BPA', 'UNITED TELEPHONE CO NW', 'WASHBURN CLINTON C & MARCIA J', 
              'WASHINGTON SCHOOL FOR THE DEAF', 'WASHINGTON SCHOOL FOR THE DEAF', 'WASHINGTON TAMMY L']

In [16]:
pubOwners = removeTerms(pubOwners, removeList)

In [17]:
privOwners = removeTerms(owners, pubOwners)

In [18]:
pubOwners = pubOwners + [' ',  'BONNEVILLE POWER ADMINISTRATION', 'CDID #1', 'CDID #15', 'CDID #2', 'CDID #3', 'COWLITZ CO FIRE DIST #1', 'COWLITZ CO FIRE DIST #2 ETAL', 'COWLITZ CO FIRE DIST #5', 
                         'COWLITZ CO RURAL FIRE DIST#2', 'COWLITZ CO FIRE DIST #5', 'PUD #1']

In [19]:
privOwners = removeTerms(owners, pubOwners)

In [20]:
privOwners

['1115 SW 11TH AVENUE LP',
 '5959 VAN ALSTINE ASSOCIATES',
 'ACHESON JOHN & MIWA',
 'AGUILAR MICHAEL A',
 'AMPE JAMES J',
 'ANDREY V GOLDINOV',
 'ASPMO EDSEL G TRUST',
 'ASTORIA NORTHWEST HOMES INC',
 'AUSTIN JONATHON E/VICTORIA M',
 'AUTOZONE DEVELOPMENT CORP',
 'AVI ENTERPRISES LLC',
 'BAUB JESSICA L/DUFF DENNIS D JR',
 'BECK GERALD R ETUX',
 'BHRE-OREGON LLC',
 'BIG BEAMS LLC',
 'BOBEK FAMILY LLC',
 'BOEHM TIM & PAM',
 'BOOTH TROY L',
 'BROLIN-WARRENTON LLC',
 'BURBACH RANDALL S',
 'BURKLEO THOMAS O',
 'BURLINGTON NORTHERN INC (NP)',
 'CENTRAL CITY CONCERN',
 'CFT NV DEVELOPMENTS LLC',
 'CLATSOP COMMUNITY COLLEGE',
 'COAST JAC LLC',
 'COLUMBIA COUNTY HABITAT FOR HUMANITY',
 'COLUMBIA COUNTY HABITAT FOR HUMANITY INC',
 'COLUMBIA COUNTY LEGAL AIDE INC',
 'COLUMBIA COUNTY SELF HELP INC',
 'COLUMBIA COUNTY SELF-HELP INC',
 'COLUMBIA COUNTY SHRINE CLUB',
 'COLUMBIA LUTHERAN CHARITIES',
 'COLUMBIA WELLNESS',
 'CONTRERAS CALIXTO ETUX',
 'COSTCO WHOLESALE CORPORATION',
 'COUCH STREET LLC',


In [21]:
# find any priv owners with ' in them
badString = pubFind(privOwners, searchTerms = "'")

In [22]:
badString

["SOTKA'S EDEN VALLEY RANCH LLC"]

In [23]:
# Remove ' form any owners
with arcpy.da.UpdateCursor('interimPrivate', 'Land_Owner') as cursor:
    for row in cursor:
        if row[0] in badString:
           
            row[0] = row[0].replace("'", "")
        cursor.updateRow(row)
            

In [24]:
# Make sure all ' gone
owners = unique_values('interimPrivate', 'Land_Owner')
pubOwners = pubFind(owners)
pubOwners = narrowList(pubOwners)
removeList = ['CENTRAL CITY CONCERN','COLUMBIA COUNTY HABITAT FOR HUMANITY', 'COLUMBIA COUNTY SHRINE CLUB', 'COWLITZ COUNTY CASA',
             'FIRST ASSEMBLY OF GOD COWLITZ COUNTY', 'OCF JOSEPH E WESTON PUBLIC FOUNDATION', 'SAFE OF COLUMBIA COUNTY', 'UNITED STATES GYPSUM', 
              'UNITED STATES GYPSUM', 'UNITED STATES OF AMERICA BPA', 'UNITED STATES OF AMERICA BPA', 'UNITED TELEPHONE CO NW', 'WASHBURN CLINTON C & MARCIA J', 
              'WASHINGTON SCHOOL FOR THE DEAF', 'WASHINGTON SCHOOL FOR THE DEAF', 'WASHINGTON TAMMY L']
pubOwners = removeTerms(pubOwners, removeList)
pubOwners = pubOwners + [' ', 'CDID #1', 'CDID #15', 'CDID #2', 'CDID #3', 'COWLITZ CO FIRE DIST #1', 'COWLITZ CO FIRE DIST #2 ETAL', 'COWLITZ CO FIRE DIST #5', 
                         'COWLITZ CO RURAL FIRE DIST#2', 'COWLITZ CO FIRE DIST #5']
privOwners = removeTerms(owners, pubOwners)
badString = pubFind(privOwners, searchTerms = "'")
print(badString)

[]


In [25]:
# Create layer from list of private owners
sql = sqlClause(privOwners, 'privatized', 'Land_Owner')
privatizedLands = arcpy.SelectLayerByAttribute_management('interimPrivate', where_clause = sql)
arcpy.CopyFeatures_management(privatizedLands, 'privatizedLandsPoints')



<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\privatizedLandsPoints'>

In [26]:
# Create selection of lands that contain privatized layers points
# and spatial join together to get polygons that have been privatized since last time
privatePolygons = arcpy.SelectLayerByLocation_management('county_merge', 'CONTAINS', 'privatizedLandsPoints')
arcpy.SpatialJoin_analysis(privatePolygons, 'privatizedLandsPoints', 'privatizedLands', match_option = 'COMPLETELY_CONTAINS')



<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\privatizedLands'>

In [27]:
fieldList = arcpy.ListFields(privatizedLands)

for field in fieldList:
    print(field.name)

removeFields = [i for i in fieldList if (i.name == 'Join_Count' or i.name == 'TARGET_FID' or i.name == 'Land_Owner' or i.name == 'PublicLand')]


OBJECTID
Shape
Join_Count
TARGET_FID
PublicLand
Land_Owner
ORIG_FID
owner
pub_pvt


In [28]:
for field in removeFields:
 
    arcpy.DeleteField_management('privatizedLands', field.name)
    